#  Install the required libraries

In [ ]:
!pip install matplotlib
!pip install pandas
!pip install re
!pip install numpy
!pip install collections
!pip install math

!pip install nltk
!pip install NRCLex
!python -m textblob.download_corpora
!pip install beautifulsoup4
!pip install tqdm
!pip install gensim
!pip install sklearn



# Make necessary imports

In [35]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report

## Gather the test jokes and extract hand-crafted features

In [29]:
import sys
sys.path.append('../code/')

from make_embed import make_embeddings
from joke_scraper import scraped_jokes
from semantic_features import clean_data, get_pos_tagged_sentences, path_similarity

### Gather the syntatic and SSE based features along with the sematic features except 'closest_path', and 'farthest_path'

In [69]:
data = pd.DataFrame({'text' : scraped_jokes})

emotions_list = ['all']
# emotions_list=['surprise', 'trust', 'anticipation']

sse_list = ['all']
#sse_list=['vp_count', 'avg_VP_len', 'rpnv', 'phrase_length_ratios_NP', 'phrase_length_ratios_VP']

#semantic_list = ['all']
semantic_list = ['disconnection', 'repitition', 'sense_combination', 'num_alliteration', 'num_rhymes', 'max_alliteration', 'max_rhymes']

filtered_emotion_features, filtered_sse_features, filtered_semantic_features= make_embeddings(data, emotions_list=emotions_list, sse_list=sse_list, semantic_list = semantic_list)
disconnection, repitition, sense_combination, num_alliteration, num_rhymes, max_alliteration, max_rhymes = filtered_semantic_features[0], filtered_semantic_features[1], filtered_semantic_features[2], filtered_semantic_features[3], filtered_semantic_features[4], filtered_semantic_features[5], filtered_semantic_features[6]


  0%|          | 0/101 [00:00<?, ?it/s]

Processing Sentences: 100%|██████████| 101/101 [00:00<00:00, 703.38it/s]


Getting POS Tags for each sentence


100%|██████████| 101/101 [00:00<00:00, 1474.75it/s]


Getting POS Tags Lists each sentence


100%|██████████| 101/101 [00:00<00:00, 50491.62it/s]


In [70]:
print(np.array(filtered_emotion_features).shape)

(101, 10)


In [71]:
print(np.array(filtered_sse_features).shape)

(101, 14)


In [73]:
sem_feature_len = len(filtered_semantic_features)
filtered_semantic_features = np.array(filtered_semantic_features)


### Extract the closest_path and the farthest_path features if required

In [74]:
sense_closest_features, sense_farthest_features = [],  []
if semantic_list == ['all'] or 'closest_path' in semantic_list or 'farthest_path' in semantic_list:
    words_list = clean_data(data['text'])
    pos_sentences = get_pos_tagged_sentences(words_list)
    sense_closest_features, sense_farthest_features = path_similarity(pos_sentences)
    
sense_closest_features, sense_farthest_features = np.array(sense_closest_features), np.array(sense_farthest_features)
sense_closest_features.shape, sense_farthest_features.shape

((0,), (0,))

## Concate all the handcrafted features to get the final list

In [75]:
if sense_closest_features and sense_farthest_features:
    final_features = np.concatenate((filtered_emotion_features, filtered_sse_features, filtered_semantic_features.reshape(101, sem_feature_len), sense_closest_features.reshape(-1, 1), sense_farthest_features.reshape(-1, 1)), axis=1)
elif sense_closest_features:
    final_features = np.concatenate((filtered_emotion_features, filtered_sse_features, filtered_semantic_features.reshape(101, sem_feature_len), sense_closest_features.reshape(-1, 1)), axis=1)
elif sense_farthest_features:
    final_features = np.concatenate((filtered_emotion_features, filtered_sse_features, filtered_semantic_features.reshape(101, sem_feature_len), sense_farthest_features.reshape(-1, 1)), axis=1)
else:
    final_features = np.concatenate((filtered_emotion_features, filtered_sse_features, filtered_semantic_features.reshape(101, sem_feature_len)), axis=1)

final_features.shape

(101, 31)

# Load the model


## TODO : Tanisha